### Extract From Database

In [1]:
import sys
import pandas as pd
import numpy as np
import pymysql
import xgboost

In [2]:
db = pymysql.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     password="12345678") # name of the data base  

In [3]:
#sql = ("""SELECT OFFENSE_CODE, OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION, OFFENSE_DETAILS,YEAR, MONTH,DAY_OF_WEEK,HOUR,UCR_PART,
#STREET,LATITUDE,LONGITUDE,LOCATION	FROM BOSTON_CRIME.OFFENSE,BOSTON_CRIME.OFFENSE_LOCATION,BOSTON_CRIME.INCIDENT
#WHERE OFFENSE.OFFENSE_ID=INCIDENT.OFFENSE_ID AND OFFENSE_LOCATION.LOCATION_ID=INCIDENT.LOCATION_ID""")

In [4]:
sql_1 = ("""SELECT MONTH,DAY_OF_WEEK,HOUR, DISTRICT, STREET,UCR_PART, OFFENSE_CODE_GROUP	FROM BOSTON_CRIME.OFFENSE,BOSTON_CRIME.OFFENSE_LOCATION,BOSTON_CRIME.INCIDENT
WHERE OFFENSE.OFFENSE_ID=INCIDENT.OFFENSE_ID AND OFFENSE_LOCATION.LOCATION_ID=INCIDENT.LOCATION_ID""")

In [5]:
#df_BostonCrime = pd.read_sql(sql,db)

In [6]:
df_S = pd.read_sql(sql_1,db)

In [7]:
#df_BostonCrime.shape

In [8]:
df_S.shape

(304268, 7)

In [9]:
df_S.head()

,MONTH,DAY_OF_WEEK,HOUR,DISTRICT,STREET,UCR_PART,OFFENSE_CODE_GROUP
0,10,1,19,D14,COLWELL AVE,3,MOTOR VEHICLE ACCIDENT RESPONSE
1,9,3,16,E18,DRAPER RD,3,PROPERTY LOST
2,9,1,12,E5,ALTACREST RD,2,FRAUD
3,9,3,8,A1,SHAWMUT ST,4,AUTO THEFT RECOVERY
4,9,3,9,E5,RAILROAD ST,1,AUTO THEFT


In [10]:
#df_BostonCrime.head()

In [11]:
## For Feature Selection
from sklearn.feature_selection import RFE

## For Splitting data into Test and Train sets
from sklearn.model_selection import train_test_split

## For Classfication Models
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from xgboost import XGBClassifier

## For Accuracy
import statsmodels
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, precision_score

## For encoding
from sklearn.preprocessing import LabelEncoder

In [12]:
labelencoder = LabelEncoder()

In [13]:
for col in ['STREET','OFFENSE_CODE_GROUP','DISTRICT']:    
    df_S.loc[:, col] = LabelEncoder().fit_transform(df_S[col])
 
X = df_S.drop(columns=['UCR_PART',])

y = df_S['UCR_PART'].to_frame()

In [14]:
#train_pct_index = int(0.9 * len(df_S))
#X_train, X_test = X[:train_pct_index], X[train_pct_index:]
#y_train, y_test = y[:train_pct_index], y[train_pct_index:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [15]:
classifier = KNeighborsClassifier(n_neighbors=5)

classifier.fit(X_train, y_train)

/Users/jasminekaur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [16]:
y_pred_knn = classifier.predict(X_test)

In [17]:
print(confusion_matrix(y_test, y_pred_knn))

[[14876  1190  1841    40]
 [  986 20440  6615    13]
 [ 2288  5112 37524     2]
 [  296    24    16    18]]


In [18]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           1       0.81      0.83      0.82     17947
           2       0.76      0.73      0.75     28054
           3       0.82      0.84      0.83     44926
           4       0.25      0.05      0.08       354

    accuracy                           0.80     91281
   macro avg       0.66      0.61      0.62     91281
weighted avg       0.80      0.80      0.80     91281



In [19]:
print ("Accuracy of KNN is {}".format(accuracy_score(y_test,y_pred_knn)*100))

Accuracy of KNN is 79.81726755841851


In [20]:
# Logistic Regression
logisticRegr = LogisticRegression()

In [21]:
logisticRegr.fit(X_train, y_train)

/Users/jasminekaur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jasminekaur/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jasminekaur/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
y_pred_lr = logisticRegr.predict(X_test)

In [23]:
print(confusion_matrix(y_test, y_pred_lr))

[[ 3876     0 14071     0]
 [ 6974     9 21071     0]
 [  421     1 44504     0]
 [  330     0    24     0]]


In [24]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           1       0.33      0.22      0.26     17947
           2       0.90      0.00      0.00     28054
           3       0.56      0.99      0.71     44926
           4       0.00      0.00      0.00       354

    accuracy                           0.53     91281
   macro avg       0.45      0.30      0.24     91281
weighted avg       0.62      0.53      0.40     91281



/Users/jasminekaur/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
print ("Accuracy of LogReg is {}".format(accuracy_score(y_test,y_pred_lr)*100))

Accuracy of LogReg is 53.01103186862546
